In [1]:
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from PIL import Image
from tqdm import tqdm
from function import *
from models import *
import os
import pywt

!nvidia-smi
print("CUDA Available : ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Tue Nov 14 12:50:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.01                 Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:2B:00.0  On |                  N/A |
| 44%   50C    P8              40W / 350W |  19833MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Preprocessing
test_tfm = transforms.Compose([
    transforms.Resize([160, 160]),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
toPIL = transforms.ToPILImage()
totensor = transforms.ToTensor()
normal = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [3]:
# Casia
train_set = DatasetFolder("../../dataset/casia100_dataset/casia", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
#train_set_size = int(len(train_set) * 0.8)
#train_set, valid_set = torch.utils.data.random_split(train_set, [train_set_size, len(train_set) - train_set_size])

train_loader = DataLoader(train_set, batch_size=128, shuffle=False, num_workers=0)
#test_loader = DataLoader(valid_set, batch_size=128, shuffle=False, num_workers=0)

Debugger :  class -> idx 
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '91': 91, '92': 92, '93': 93, '94': 94, '95': 95, '96': 96, '97': 97, '98': 98, '99

In [4]:
model = InceptionResnetV1(pretrained='casia-webface',classify=None).to(device)
model.logits = nn.Linear(in_features = model.logits.in_features, out_features = 100)
model.last_linear = nn.Linear(in_features = model.last_linear.in_features, out_features = 100, bias = True)
model.last_bn = nn.BatchNorm1d(num_features = 100)
checkpoint = torch.load('pretrain/facenet_casia.pth')
model.load_state_dict(checkpoint)
#cudnn.benchmark = True
model.to(device)
adversary = attack(model, 'fgsm')


path = ['adv_dwt', 'dwt']
level = 10


for _dir_ in path:
    if not os.path.isdir(_dir_):
        os.mkdir(_dir_)
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        
        result = imgs
        if 'adv' in _dir_:
            imgs, eta = adversary.perturb(imgs, labels)
            result = eta
        result = result.cpu()
        
        
        # DWT
        result, (LL, LH, HL, HH) = discrete_wavelet(result, 'db2', 1)

        #CWT
        #result = continuous_wavelet(imgs, 'mexh', level, cuda = False)
        
        # DWT
        for i in range(imgs.size(0)):
            img = toPIL(imgs[i])
            img.save(f'{_dir_}/img_{i}.png')
            eta_i = toPIL(eta[i])
            eta_i.save(f'{_dir_}/eta_{i}.png')
            ll = toPIL(LL[i])
            ll.save(f'{_dir_}/ll_{i}.png')
            lh = toPIL(LH[i]).convert('RGB')
            lh.save(f'{_dir_}/lh_{i}.png')
            hl = toPIL(HL[i]).convert('RGB')
            hl.save(f'{_dir_}/hl_{i}.png')
            hh = toPIL(HH[i]).convert('RGB')
            hh.save(f'{_dir_}/hh_{i}.png')
            inverse = toPIL(result[i])
            inverse.save(f'{_dir_}/inverse_{i}.png')
        
        # CWT
        #for i in range(imgs.size(0)):
        #    for j in range(level):
        #        img = toPIL(torch.clamp(result[j][i], 0, 1)).convert('RGB')
        #        img.save(f'{_dir_}/img_{i}_level{j}.png')
        break


  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]